In [1]:
# Example： 從 Request 到 BeatifulSoup

import requests
from bs4 import BeautifulSoup

url = 'https://scholar.google.com.tw/citations?user=g-_ZXGsAAAAJ&hl=zh-TW&oi=ao'
r = requests.get(url)
soup = BeautifulSoup(r.text)



In [10]:
import datetime
now = datetime.datetime.now()
currentTime = now.strftime("%Y-%m-%d %H:%M:%S")
print (currentTime)

2021-09-06 12:47:19


In [33]:
content = ''.join(soup.find('meta', property = 'og:image')['content']) #list to string
id = content.split('user=')[1].split('&citpid=1')[0] 
id

'g-_ZXGsAAAAJ'

In [32]:

 #personal detail
 #personal detail
info = {}

#id
content = ''.join(soup.find('meta', property = 'og:image')['content']) #list to string
id = content.split('user=')[1].split('&citpid=1')[0] 

d = soup.find('div', id = 'gsc_prf_i')

#name
info['name'] = d.find('div', id = 'gsc_prf_in').text

#university
info['university'] = d.find('a', class_ = 'gsc_prf_ila').text

#email
email = d.find('div', id = 'gsc_prf_ivh').text.split(' ')[1]
info['email'] = email

#picture
info['picture'] = soup.find('div', id = 'gsc_prf_pua').find('img')['src'] 


info


{'name': 'Anil K. Jain',
 'university': 'Michigan State University',
 'email': 'cse.msu.edu',
 'picture': 'https://scholar.googleusercontent.com/citations?view_op=view_photo&user=g-_ZXGsAAAAJ&citpid=1'}

In [33]:
citeBy = {}
citations = {}
h_index = {}
i10_index = {}

def cited(status, value):
    
    if status / 2 < 1:        
        if status % 2 == 0:
            citations['All'] = value
        else:
            citations['Since2016'] = value
        citeBy['citations'] = citations
        
    if status / 2 < 2:
        if status % 2 == 0:
            h_index['All'] = value
        else:
            h_index['Since2016'] = value
        citeBy['h_index'] = h_index
        
    if status / 2 < 3:
        if status % 2 == 0:
            i10_index['All'] = value
        else:
            i10_index['Since2016'] = value
        citeBy['i10_index'] = i10_index


In [34]:

count_d = 0
for d in soup.find_all('td', class_='gsc_rsb_std'):

    cited(count_d, d.text)
    
    count_d = count_d + 1
citeBy

{'citations': {'All': '228713', 'Since2016': '72308'},
 'h_index': {'All': '199', 'Since2016': '119'},
 'i10_index': {'All': '731', 'Since2016': '501'}}

In [35]:

label = []
for p in soup.find_all('a', class_='gsc_prf_inta gs_ibl'):

    label.append(p.text)
info['label'] = label
label

['Biometrics',
 'Computer vision',
 'Pattern recognition',
 'Machine learning',
 'Image processing']

In [36]:
results = []
infos = {}
infos['id'] = id
infos['updateTime'] = currentTime
infos['personalData'] = info
infos['cited'] = citeBy

results.append(infos)

results

[{'id': 'g-_ZXGsAAAAJ',
  'updateTime': '2021-09-06 12:47:19',
  'personalData': {'name': 'Anil K. Jain',
   'university': 'Michigan State University',
   'email': 'cse.msu.edu',
   'picture': 'https://scholar.googleusercontent.com/citations?view_op=view_photo&user=g-_ZXGsAAAAJ&citpid=1',
   'label': ['Biometrics',
    'Computer vision',
    'Pattern recognition',
    'Machine learning',
    'Image processing']},
  'cited': {'citations': {'All': '228713', 'Since2016': '72308'},
   'h_index': {'All': '199', 'Since2016': '119'},
   'i10_index': {'All': '731', 'Since2016': '501'}}}]

In [37]:
import json
jsonStr = json.dumps(results)
print(jsonStr)


[{"id": "g-_ZXGsAAAAJ", "updateTime": "2021-09-06 12:47:19", "personalData": {"name": "Anil K. Jain", "university": "Michigan State University", "email": "cse.msu.edu", "picture": "https://scholar.googleusercontent.com/citations?view_op=view_photo&user=g-_ZXGsAAAAJ&citpid=1", "label": ["Biometrics", "Computer vision", "Pattern recognition", "Machine learning", "Image processing"]}, "cited": {"citations": {"All": "228713", "Since2016": "72308"}, "h_index": {"All": "199", "Since2016": "119"}, "i10_index": {"All": "731", "Since2016": "501"}}}]


In [41]:
jsonFile = open("jsonFile/personalData.json", "w")
jsonFile.write(jsonStr)
jsonFile.close()

In [41]:
#firebase connect test

In [42]:
%%capture
pip install --upgrade firebase-admin 


In [43]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

if not firebase_admin._apps:
    cred = credentials.Certificate("service-account.json")
    firebase_admin.initialize_app(cred)
    
db = firestore.client()

In [44]:
users_ref = db.collection(u'cguscholar').document(u'faE3_ksAAAAJ').collection(u'updata_time')
docs = users_ref.stream()

for doc in docs:
    print(f'{doc.id} => {doc.to_dict()}')

1 => {'citations': 116830, 'timestamp': DatetimeWithNanoseconds(2021, 8, 30, 6, 0, tzinfo=<UTC>)}
2 => {'time': DatetimeWithNanoseconds(2021, 8, 3, 5, 29, tzinfo=<UTC>), 'citations': '116900'}


In [ ]:
class UploadJsonFileToFirestore:
    def __init__(self) -> None:
        # Get class running time
        self.start = timeit.default_timer()
        # Check to make sure the command line arguements 
        # are atleast 3 arguements
        if len(sys.argv[1:]) != 3:
            print(f'ERROR: Check your command line arguments!,\n 3 arguements expected [file=filepath, method=[set or add], collectionname=[firestore collection name]')
            return None
        
        # Initialize instance variables
        self.json_data = sys.argv[1:][0]
        self.method = sys.argv[1:][1]
        self.collectionname = sys.argv[1:][2]
    
    def __str__(self) -> str:
        return (f'Uploading ****{self.file}***** JSON items to firestore!')
    
    # Firestore upload method getter method
    @property
    def method(self):
        return self._method
    
    # Firestore upload method setter method
    @method.setter
    def method(self, val):
        if val == 'set' or val == 'add':
            self._method = val
        else:
            print(f'Wrong method {val}, use set or add')
    
    # Get Json file path property
    @property
    def json_data(self):
        return self._json_data
    
    # Set and process Json file path property
    @json_data.setter
    def json_data(self, val):
        if val:
            try:
                # Opening JSON file
                f = open(val,)
                
                # returns JSON object as a dictionary
                data = json.load(f)
                
                # make sure to close file
                f.close()
                self._json_data = data
            except Exception as e:
                print(f'FILE EXCEPTION: {str(e)}')
        else:
            print(f'Wrong file path {val}')

    # Main class method to populate firestore 
    # With the said data
    def upload(self):
        if  self.json_data and self.method:
           
            # Iterating through the json list
            for idx, item in enumerate(self.json_data):
                '''
                 START FOR JUST FOR DEMO REASONS
                '''
                from pygments import highlight
                from pygments.lexers import JsonLexer
                from pygments.formatters import TerminalFormatter
              
                json_str = json.dumps(item, indent=4, sort_keys=True)
                print(highlight(json_str, JsonLexer(), TerminalFormatter()))
                '''
                 END FOR JUST FOR DEMO REASONS
                '''
             
                if self.method == 'set':
                    self.set(item)
                else:
                    self.add(item)
                # Successfully got to end of data;
                # print success message
                if idx == len(self.json_data)-1:
                    # All the program statements
                    stop = timeit.default_timer()
                    print('**************************\n****SUCCESS UPLOAD*****\n**************************')
                    print("Time taken "+str(stop - self.start))
    
    # Collection Add method
    # Adds all data under a collection
    # With firebase firestore auto generated IDS
    def add(self, item):
        return db.collection(self.collectionname).add(item)
    
    # Collection document set method
    # Adds all data under a collection
    # With custom document IDS 
    def set(self, item):
        return db.collection(self.collectionname).document(str(item['id'])).set(item)

uploadjson = UploadJsonFileToFirestore()
uploadjson.upload()      